In [ ]:
!pip install pillow-heif tensorflow keras opencv-python-headless pandas --quiet

from google.colab import drive
import os, cv2, numpy as np, pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from PIL import Image
import pillow_heif
import matplotlib.pyplot as plt

# ==========================================
# STEP 1: Mount Drive
# ==========================================
drive.mount('/content/drive')

base_path = "/content/drive/MyDrive/faces_dataset"
group_folder = os.path.join(base_path, "group_photo")
model_path = os.path.join(base_path, "trained_face_model_v3.h5")

# ==========================================
# STEP 2: Convert HEIC → JPG (if any)
# ==========================================
def convert_heic_to_jpg(folder):
    pillow_heif.register_heif_opener()
    for file in os.listdir(folder):
        if file.lower().endswith(".heic"):
            heic_path = os.path.join(folder, file)
            jpg_path = heic_path.replace(".HEIC", ".jpg").replace(".heic", ".jpg")
            try:
                img = Image.open(heic_path)
                img.save(jpg_path, "JPEG")
                os.remove(heic_path)
                print(f"✅ Converted {file} → {os.path.basename(jpg_path)}")
            except Exception as e:
                print(f"❌ Error converting {file}: {e}")

for d in os.listdir(base_path):
    f = os.path.join(base_path, d)
    if os.path.isdir(f):
        convert_heic_to_jpg(f)

# ==========================================
# STEP 3: Face alignment helper
# ==========================================
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def align_and_normalize(img):
    """Detect and crop face, equalize lighting."""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
    if len(faces) > 0:
        faces = sorted(faces, key=lambda x: x[2]*x[3], reverse=True)
        x, y, w, h = faces[0]
        face = img[y:y+h, x:x+w]
        yuv = cv2.cvtColor(face, cv2.COLOR_BGR2YUV)
        yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
        return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    return img

# ==========================================
# STEP 4: Load faces from folders
# ==========================================
def load_faces(base_path):
    imgs, labels, names = [], [], []
    person_dirs = sorted([d for d in os.listdir(base_path)
                          if os.path.isdir(os.path.join(base_path, d)) and d != "group_photo"])
    for idx, person in enumerate(person_dirs):
        p_path = os.path.join(base_path, person)
        names.append(person)
        for f in os.listdir(p_path):
            if f.lower().endswith(('.jpg','.jpeg','.png')):
                img_path = os.path.join(p_path, f)
                try:
                    img = cv2.imread(img_path)
                    if img is None: continue
                    face = align_and_normalize(img)
                    face = cv2.resize(face, (100, 100))
                    imgs.append(face)
                    labels.append(idx)
                except Exception as e:
                    print(f"Error loading {img_path}: {e}")
    print(f"✅ Loaded {len(imgs)} images across {len(names)} classes: {names}")
    return np.array(imgs), np.array(labels), names

X, y, names = load_faces(base_path)
if len(X) == 0:
    print("❌ No training data found.")
    raise SystemExit

X = X.astype('float32')/255.0
y = to_categorical(y, len(names))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# STEP 5: Build & Train Model
# ==========================================
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(100,100,3)),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(names), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=30, batch_size=8, validation_data=(X_val, y_val))
model.save(model_path)
print("✅ Model trained & saved at:", model_path)

# ==========================================
# STEP 6: Skin-tone filtering helper
# ==========================================
def skin_mask(bgr_img):
    """Return binary mask for skin-like pixels using HSV & YCrCb thresholds."""
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2YCrCb)

    # HSV skin range
    lower_hsv = np.array([0, 30, 60], dtype=np.uint8)
    upper_hsv = np.array([20, 150, 255], dtype=np.uint8)
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)

    # YCrCb skin range
    lower_ycrcb = np.array([0, 133, 77], dtype=np.uint8)
    upper_ycrcb = np.array([255, 173, 127], dtype=np.uint8)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)

    combined_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)
    combined_mask = cv2.medianBlur(combined_mask, 5)
    return combined_mask

# ==========================================
# STEP 7: Detect faces in group photo
# ==========================================
def detect_faces_in_group(image_path, model, names, threshold=0.5):
    img = cv2.imread(image_path)
    if img is None:
        print("❌ Could not load image:", image_path)
        return None

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1,
                                          minNeighbors=6, minSize=(50,50))
    results = []

    for (x, y, w, h) in faces:
        face_img = img[y:y+h, x:x+w]

        # Step 1: Check if region is mostly skin-colored
        mask = skin_mask(face_img)
        skin_ratio = np.sum(mask > 0) / (mask.size)
        if skin_ratio < 0.15:  # skip if not enough skin pixels
            continue

        # Step 2: Classify
        face_resized = cv2.resize(face_img, (100, 100)) / 255.0
        face_resized = np.expand_dims(face_resized, axis=0)
        preds = model.predict(face_resized, verbose=0)
        label_idx = np.argmax(preds)
        prob = preds[0][label_idx]

        # Step 3: Only draw if confidence >= threshold
        if prob >= threshold:
            label = names[label_idx]
            color = (0, 255, 0)
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 3)
            cv2.putText(img, f"{label} ({prob:.2f})", (x, y-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
            results.append([x, y, w, h, label, round(float(prob), 4)])
        # Skip drawing if prob < threshold → no box shown

    output_img_path = "/content/group_photo_predictions_v3.jpg"
    cv2.imwrite(output_img_path, img)
    df = pd.DataFrame(results, columns=["x","y","w","h","label","probability"])
    df.to_csv("predictions_v3.csv", index=False)
    print("✅ Saved annotated image at:", output_img_path)
    print("✅ Predictions saved to predictions_v3.csv")
    return df

# ==========================================
# STEP 8: Run detection
# ==========================================
group_photo_path = os.path.join(group_folder, "IMG_1473.jpg")
preds_df = detect_faces_in_group(group_photo_path, model, names, threshold=0.5)
print("\nPredictions Summary:")
print(preds_df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 